## Question 2 ##
# (d) #

In [ ]:
pragma solidity >=0.5.1 <0.6.0;

library SafeMath {
    function add(uint256 a, uint256 b) internal pure returns (uint256) {
        uint256 c = a + b; require(c >= a); return c;
    }
    function sub(uint256 a, uint256 b) internal pure returns (uint256) {
        require(b <= a); return a - b;
    }
}

contract FixedVoting {
    using SafeMath for uint256;

    mapping (address => uint256) public remainingVotes;
    uint256[] public candidates;
    address owner;
    bool hasEnded = false;

    modifier notEnded() { require(!hasEnded); _; }

    constructor(uint256 amountOfCandidates) public {
        candidates.length = amountOfCandidates;
        owner = msg.sender;
    }

    function buyVotes() public payable notEnded {
        require(msg.value >= 1 ether);
        uint256 votes = msg.value / 1 ether;
        uint256 refund = msg.value % 1 ether;
        remainingVotes[msg.sender] = remainingVotes[msg.sender].add(votes);
        if (refund > 0) msg.sender.transfer(refund);
    }

    function vote(uint256 _candidateID, uint256 _amountOfVotes) public notEnded {
        require(_candidateID < candidates.length);
       
        remainingVotes[msg.sender] = remainingVotes[msg.sender].sub(_amountOfVotes);
        candidates[_candidateID] = candidates[_candidateID].add(_amountOfVotes);
    }

    function payoutVotes(uint256 _amount) public notEnded {
      
        remainingVotes[msg.sender] = remainingVotes[msg.sender].sub(_amount);
        msg.sender.transfer(_amount * 1 ether);
    }

    function endVoting() public notEnded {
        require(msg.sender == owner);
        hasEnded = true;
        msg.sender.transfer(address(this).balance);
    }
}